In [ ]:
__copyright__ = "Reiner Lemoine Institut gGmbH"
__license__ = "GNU Affero General Public License Version 3 (AGPL-3.0)"
__url__ = "https://github.com/openego/eDisGo/blob/master/LICENSE"
__author__ = "gplssm, birgits, khelfen"

# A minimum working example

Following you find short examples on how to use eDisGo to set up a network and time series information for loads and generators in the network and afterwards conduct a power flow analysis and determine possible grid expansion needs and costs. Further details are provided in Usage details. Further examples can be found in the examples directory.

All following examples assume you have a ding0 grid topology (directory containing csv files, defining the grid topology) in a directory “ding0_example_grid” in the directory from where you run your example. If you do not have an example grid, you can download one here.

Aside from grid topology data you may eventually need a dataset on future installation of power plants. You may therefore use the scenarios developed in the open_eGo project that are available in the OpenEnergy DataBase (oedb) hosted on the OpenEnergy Platform (OEP). eDisGo provides an interface to the oedb using the package ego.io. ego.io gives you a python SQL-Alchemy representations of the oedb and access to it by using the oedialect, an SQL-Alchemy dialect used by the OEP.

You can run a worst-case scenario as follows:

In [ ]:
import os
import pandas as pd
from edisgo import EDisGo

### Download example grid

In [ ]:
import requests

def download_ding0_example_grid():

    # create directories to save ding0 example grid into
    ding0_example_grid_path = os.path.join(
        os.path.expanduser("~"), ".edisgo", "ding0_test_network"
    )
    os.makedirs(ding0_example_grid_path, exist_ok=True)

    # download files
    filenames = [
        "buses",
        "generators",
        "lines",
        "loads",
        "network",
        "switches",
        "transformers",
        "transformers_hvmv",
    ]

    for file in filenames:
        req = requests.get(
            "https://raw.githubusercontent.com/openego/eDisGo/features/%23261-emob-tests/tests/data/ding0_test_network_2/{}.csv".format(
                file
            )
        )
        filename = os.path.join(ding0_example_grid_path, "{}.csv".format(file))
        with open(filename, "wb") as fout:
            fout.write(req.content)


download_ding0_example_grid()
ding0_grid = os.path.join(os.path.expanduser("~"), ".edisgo", "ding0_test_network")

In [ ]:
# Set up the EDisGo object - the EDisGo object provides the top-level API for
# invocation of data import, power flow analysis, network reinforcement,
# flexibility measures, etc..
edisgo_obj = EDisGo(ding0_grid=ding0_grid)

# Import scenario for future generator park from the oedb
edisgo_obj.import_generators(generator_scenario="nep2035")

In [ ]:
# Set up feed-in and load time series (here for a worst case analysis)
edisgo_obj.set_time_series_worst_case_analysis()

# Conduct power flow analysis (non-linear power flow using PyPSA)
edisgo_obj.analyze()

In [ ]:
# Do grid reinforcement
edisgo_obj.reinforce()

# Determine costs for each line/transformer that was reinforced
costs = edisgo_obj.results.grid_expansion_costs

Instead of conducting a worst-case analysis you can also provide specific time series:

In [ ]:
# Set up the EDisGo object with generator park scenario NEP2035
edisgo_obj = EDisGo(
    ding0_grid=ding0_grid,
    generator_scenario="nep2035"
)

# Set up your own time series by load sector and generator type (these are dummy
# time series!)
timeindex = pd.date_range("1/1/2011", periods=4, freq="H")
# load time series (scaled by annual demand)
timeseries_load = pd.DataFrame(
    {"residential": [0.0001] * len(timeindex),
     "retail": [0.0002] * len(timeindex),
     "industrial": [0.00015] * len(timeindex),
     "agricultural": [0.00005] * len(timeindex)
     },
    index=timeindex)
# feed-in time series of fluctuating generators (scaled by nominal power)
timeseries_generation_fluctuating = pd.DataFrame(
    {"solar": [0.2] * len(timeindex),
     "wind": [0.3] * len(timeindex)
     },
    index=timeindex)
# feed-in time series of dispatchable generators (scaled by nominal power)
timeseries_generation_dispatchable = pd.DataFrame(
    {"biomass": [1] * len(timeindex),
     "coal": [1] * len(timeindex),
     "other": [1] * len(timeindex)
     },
    index=timeindex)

# Before you can set the time series to the edisgo_obj you need to set the time
# index (this could also be done upon initialisation of the edisgo_obj) - the time
# index specifies which time steps to consider in power flow analysis
edisgo_obj.set_timeindex(timeindex)

# Now you can set the active power time series of loads and generators in the grid
edisgo_obj.set_time_series_active_power_predefined(
    conventional_loads_ts=timeseries_load,
    fluctuating_generators_ts=timeseries_generation_fluctuating,
    dispatchable_generators_ts=timeseries_generation_dispatchable
)

# Before you can now run a power flow analysis and determine grid expansion needs,
# reactive power time series of the loads and generators also need to be set. If you
# simply want to use default configurations, you can do the following.
edisgo_obj.set_time_series_reactive_power_control()

# Now you are ready to determine grid expansion needs
edisgo_obj.reinforce()

# Determine cost for each line/transformer that was reinforced
costs = edisgo_obj.results.grid_expansion_costs

Time series for loads and fluctuating generators can also be automatically generated using the provided API for the oemof demandlib and the OpenEnergy DataBase:

In [ ]:
# Set up the EDisGo object with generator park scenario NEP2035 and time index
timeindex = pd.date_range("1/1/2011", periods=4, freq="H")
edisgo_obj = EDisGo(
    ding0_grid=ding0_grid,
    generator_scenario="nep2035",
    timeindex=timeindex
)

# Set up your own time series by load sector and generator type (these are dummy
# time series!)
# Set up active power time series of loads and generators in the grid using prede-
# fined profiles per load sector and technology type
# (There are currently no predefined profiles for dispatchable generators, wherefore
# their feed-in profiles need to be provided)
timeseries_generation_dispatchable = pd.DataFrame(
    {"biomass": [1] * len(timeindex),
     "coal": [1] * len(timeindex),
     "other": [1] * len(timeindex)
     },
    index=timeindex
)
edisgo_obj.set_time_series_active_power_predefined(
    conventional_loads_ts="demandlib",
    fluctuating_generators_ts="oedb",
    dispatchable_generators_ts=timeseries_generation_dispatchable
)

# Before you can now run a power flow analysis and determine grid expansion needs,
# reactive power time series of the loads and generators also need to be set. Here,
# default configurations are again used.
edisgo_obj.set_time_series_reactive_power_control()

# Do grid reinforcement
edisgo_obj.reinforce()

# Determine cost for each line/transformer that was reinforced
costs = edisgo_obj.results.grid_expansion_costs


# Usage details
## The fundamental data structure

In [ ]:
edisgo = edisgo_obj

In [ ]:
# Access Topology grid data container object
edisgo.topology

# Access TimeSeries data container object
edisgo.timeseries

# Access Electromobility data container object
edisgo.electromobility

# Access Results data container object
edisgo.results

# Access configuration data container object
edisgo.config

# Access all buses in MV grid and underlying LV grids
edisgo.topology.buses_df

# Access all lines in MV grid and underlying LV grids
edisgo.topology.lines_df

# Access all MV/LV transformers
edisgo.topology.transformers_df

# Access all HV/MV transformers
edisgo.topology.transformers_hvmv_df

# Access all switches in MV grid and underlying LV grids
edisgo.topology.switches_df

# Access all generators in MV grid and underlying LV grids
edisgo.topology.generators_df

# Access all loads in MV grid and underlying LV grids
edisgo.topology.loads_df

# Access all storage units in MV grid and underlying LV grids
edisgo.topology.storage_units_df

# Access MV grid
edisgo.topology.mv_grid

# Access all buses in MV grid
edisgo.topology.mv_grid.buses_df

# Access all generators in MV grid
edisgo.topology.mv_grid.generators_df

# Get list of all underlying LV grids
# (Note that MVGrid.lv_grids returns a generator object that must first be
#  converted to a list in order to view the LVGrid objects)
list(edisgo.topology.mv_grid.lv_grids)
# the following yields the same
list(edisgo.topology.lv_grids)

# Get single LV grid by providing its ID (e.g. 1) or name (e.g. "LVGrid_1")
lv_grid = edisgo.topology.get_lv_grid("LVGrid_402945")

# Access all buses in that LV grid
lv_grid.buses_df

# Access all loads in that LV grid
lv_grid.loads_df

# Get all switch disconnectors in MV grid as Switch objects
# (Note that objects are returned as a python generator object that must
#  first be converted to a list in order to view the Switch objects)
list(edisgo.topology.mv_grid.switch_disconnectors)

# Get all generators in LV grid as Generator objects
list(lv_grid.generators)

# Get graph representation of whole topology
edisgo.to_graph()

# Get graph representation for MV grid
edisgo.topology.mv_grid.graph

# Get graph representation for LV grid
lv_grid.graph

## Component time series

In [ ]:
edisgo.set_time_series_manual()
edisgo.set_time_series_worst_case_analysis()
edisgo.timeseries.timeindex_worst_cases
edisgo.set_time_series_active_power_predefined()
#edisgo.apply_charging_strategy()
edisgo.set_time_series_reactive_power_control()

## Identifying grid issues

In [ ]:
# Do non-linear power flow analysis for MV and LV grid
edisgo.analyze()

## Grid expansion

In [ ]:
# Reinforce grid due to overloading and overvoltage issues
edisgo.reinforce()

# Get costs of grid expansion
costs = edisgo.results.grid_expansion_costs

## Electromobility

In [ ]:
# ToDo

## Battery storage systems

In [ ]:
# Set up EDisGo object
edisgo = EDisGo(ding0_grid=ding0_grid)

# Get random bus to connect storage to
random_bus = edisgo.topology.buses_df.index[3]
# Add storage instance
edisgo.add_component(
    comp_type="storage_unit",
    add_ts=False,
    bus=random_bus,
    p_nom=4
)

# Set up worst case time series for loads, generators and storage unit
edisgo.set_time_series_worst_case_analysis()

In [ ]:
# Set up the EDisGo object
timeindex = pd.date_range("1/1/2011", periods=4, freq="H")
edisgo = EDisGo(
    ding0_grid=ding0_grid,
    generator_scenario="ego100",
    timeindex=timeindex
)

# Add time series for loads and generators
timeseries_generation_dispatchable = pd.DataFrame(
    {"biomass": [1] * len(timeindex),
     "coal": [1] * len(timeindex),
     "other": [1] * len(timeindex)
     },
    index=timeindex
)
edisgo.set_time_series_active_power_predefined(
    conventional_loads_ts="demandlib",
    fluctuating_generators_ts="oedb",
    dispatchable_generators_ts=timeseries_generation_dispatchable
)
edisgo.set_time_series_reactive_power_control()

# Add storage unit to random bus with time series
edisgo.add_component(
    comp_type="storage_unit",
    bus=edisgo.topology.buses_df.index[3],
    p_nom=4,
    ts_active_power=pd.Series(
        [-3.4, 2.5, -3.4, 2.5],
        index=edisgo.timeseries.timeindex),
    ts_reactive_power=pd.Series(
        [0., 0., 0., 0.],
        index=edisgo.timeseries.timeindex)
)

In [ ]:
# DOES NOT WORK
#random_bus = edisgo.topology.buses_df.index[3:13]
#edisgo.perform_mp_opf(
#    timesteps=period,
#    scenario="storage",
#    storage_units=True,
#    storage_buses=busnames,
#    total_storage_capacity=10.0,
#    results_path=results_path)

## Curtailment

In [ ]:
# DOES NOT WORK
#edisgo.perform_mp_opf(
#    timesteps=period,
#    scenario='curtailment',
#    results_path=results_path,
#    curtailment_requirement=True,
#    curtailment_requirement_series=[10, 20, 15, 0])

## Plots

In [ ]:
# plot MV grid topology on a map
edisgo_obj.plot_mv_grid_topology()

# plot grid expansion costs for lines in the MV grid and stations on a map
edisgo_obj.plot_mv_grid_expansion_costs()

# plot voltage histogram
edisgo_obj.histogram_voltage()

# ToDo: add Plotly plot option

## Results

In [ ]:
edisgo.results
#edisgo.results.save('./')